In [1]:
import pandas as pd
df = pd.read_csv('股票K线数据_90天.csv', parse_dates=['日期']) 

# 按照日期降序排列（从最近开始）
df = df.sort_values(by='日期', ascending=False).reset_index(drop=True)

# 计算向前的滚动平均（从当前行向后n天）
df['MA_5'] = df['收盘价'].rolling(window=5, min_periods=5).mean()
df['MA_10'] = df['收盘价'].rolling(window=10, min_periods=10).mean()
df['MA_20'] = df['收盘价'].rolling(window=20, min_periods=20).mean()

# 由于20日均线最短，所以只保留有20日均线的部分
df_aligned = df[df['MA_20'].notna()].copy().reset_index(drop=True)

# 只保留日期和三条均线列
result = df_aligned[['日期', 'MA_5', 'MA_10', 'MA_20']]
print(result)

               日期    MA_5   MA_10    MA_20
0      2025-04-17  29.906  21.307  19.5495
1      2025-04-17  25.382  20.203  19.2700
2      2025-04-17  34.746  25.663  22.3410
3      2025-04-17  23.168  25.584  22.1415
4      2025-04-17  31.620  29.475  24.3000
...           ...     ...     ...      ...
618975 2024-08-07   1.400   1.383   1.4110
618976 2024-08-06   1.364   1.379   1.3955
618977 2024-08-06   1.336   1.389   1.3995
618978 2024-08-05   1.332   1.367   1.3835
618979 2024-08-02   1.320   1.348   1.3645

[618980 rows x 4 columns]


In [ ]:

import pandas as pd

df = pd.read_csv('股票K线数据_90天.csv', parse_dates=['日期']) 
df['股票代码'] = df['股票代码'].astype(str).str.replace(r'^[01]\.', '', regex=True)
df['股票代码'] = df['股票代码'].astype(str).str.ljust(6, "0")
def calculate_moving_averages(data: pd.DataFrame, stock_code: str) -> pd.DataFrame:
    """
    输入：
        data: 包含多只股票的历史数据，必须包含['日期', '股票代码', '收盘价']
        stock_code: 股票代码（如 '0.001300'）

    输出：
        包含该股票的日期、5日、10日、20日均线的DataFrame，按照20日均线对齐
    """

    # 过滤指定股票的数据
    df = data[data['股票代码'] == stock_code].copy()

    # 日期格式转换并从最近日期开始排序
    df['日期'] = pd.to_datetime(df['日期'])
    df = df.sort_values(by='日期').reset_index(drop=True)

    # 计算均线（从当前日期向过去滚动）
    df['MA_5'] = df['收盘价'].rolling(window=5, min_periods=5).mean()
    df['MA_10'] = df['收盘价'].rolling(window=10, min_periods=10).mean()
    df['MA_20'] = df['收盘价'].rolling(window=20, min_periods=20).mean()

    # 对齐：只保留有20日均线的数据
    df_aligned = df[df['MA_20'].notna()].copy().reset_index(drop=True)

    # 只返回感兴趣的列
    return df_aligned[['日期', 'MA_5', 'MA_10', 'MA_20']]


d = calculate_moving_averages(data=df, stock_code='600519')

d

,日期,MA_5,MA_10,MA_20
0,2024-11-18,1545.620,1561.381,1541.7460
1,2024-11-19,1538.564,1557.874,1541.6920
2,2024-11-20,1531.826,1555.255,1540.5575
3,2024-11-21,1526.092,1544.854,1540.2040
4,2024-11-22,1515.856,1534.639,1537.6525
...,...,...,...,...
96,2025-04-11,1540.804,1552.415,1577.5490
97,2025-04-14,1551.202,1549.093,1573.7480
98,2025-04-15,1553.802,1548.793,1569.7550
99,2025-04-16,1557.428,1549.108,1566.5135


In [10]:
import pandas as pd

df = pd.read_csv('股票K线数据_90天.csv', parse_dates=['日期']) 
df['股票代码'] = df['股票代码'].astype(str).str.replace(r'^[01]\.', '', regex=True)
df['股票代码'] = df['股票代码'].astype(str).str.ljust(6, "0")

def calculate_all_stocks_averages(data: pd.DataFrame) -> pd.DataFrame:
    """
    输入：
        data: 包含多只股票的历史数据，必须包含 ['日期', '股票代码', '收盘价']
        
    输出：
        包含所有股票的 ['日期', '股票代码', 'MA_5', 'MA_10', 'MA_20']，并按20日均线对齐
    """
    results = []

    # 获取所有股票代码的去重列表
    stock_codes = data['股票代码'].unique()

    for stock_code in stock_codes:
        df_stock = data[data['股票代码'] == stock_code].copy()
        df_stock['日期'] = pd.to_datetime(df_stock['日期'])
        df_stock = df_stock.sort_values(by='日期').reset_index(drop=True)

        # 计算滚动均线
        df_stock['MA_5'] = df_stock['收盘价'].rolling(window=5, min_periods=5).mean()
        df_stock['MA_10'] = df_stock['收盘价'].rolling(window=10, min_periods=10).mean()
        df_stock['MA_20'] = df_stock['收盘价'].rolling(window=20, min_periods=20).mean()

        # 仅保留有完整20日均线的数据
        df_stock = df_stock[df_stock['MA_20'].notna()].copy()
        df_stock['股票代码'] = stock_code

        results.append(df_stock[['日期', '股票代码', 'MA_5', 'MA_10', 'MA_20']])

    # 合并所有股票结果
    df_result = pd.concat(results, ignore_index=True)

    # 日期降序排列
    df_result = df_result.sort_values(by='日期', ascending=False).reset_index(drop=True)

    return df_result
df_all = calculate_all_stocks_averages(data=df)
df_all.head()

,日期,股票代码,MA_5,MA_10,MA_20
0,2025-04-17,300337,10.462,10.275,11.2980
1,2025-04-17,603836,14.598,14.390,13.5235
2,2025-04-17,002328,5.182,5.222,5.7430
3,2025-04-17,000622,2.260,2.394,2.5475
4,2025-04-17,300344,8.748,8.553,9.8830


In [12]:
df_all.to_csv('所有股票均线.csv')

In [13]:
df_all

,日期,股票代码,MA_5,MA_10,MA_20
0,2025-04-17,300337,10.462,10.275,11.2980
1,2025-04-17,603836,14.598,14.390,13.5235
2,2025-04-17,002328,5.182,5.222,5.7430
3,2025-04-17,000622,2.260,2.394,2.5475
4,2025-04-17,300344,8.748,8.553,9.8830
...,...,...,...,...,...
520719,2024-09-03,600289,1.388,1.306,1.2815
520720,2024-09-02,600289,1.330,1.277,1.2715
520721,2024-09-02,4e-050,1.306,1.242,1.2870
520722,2024-08-30,4e-050,1.242,1.215,1.2795
